## ONNX graph, single or double floats

The notebook shows discrepencies obtained by using double floats instead of single float in two cases. The second one involves [GaussianProcessRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html).

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Simple case of a linear regression

A linear regression is simply a matrix multiplication followed by an addition: $Y=AX+B$. Let's train one with [scikit-learn](https://scikit-learn.org/stable/).

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
data = load_boston()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = LinearRegression()
clr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [3]:
clr.score(X_test, y_test)

0.6432477795844672

In [4]:
clr.coef_

array([-1.00262697e-01,  4.85344897e-02,  3.16187943e-02,  2.27453589e+00,
       -2.02674809e+01,  4.08686662e+00, -6.96395624e-03, -1.60618842e+00,
        2.87138364e-01, -1.28566669e-02, -9.19579951e-01,  4.59081604e-03,
       -5.09259407e-01])

In [5]:
clr.intercept_

38.1740039546269

Let's predict with *scikit-learn* and *python*.

In [6]:
ypred = clr.predict(X_test)
ypred[:5]

array([17.56231959, 18.92593785, 26.14022707, 22.51835039, 22.76141885])

In [7]:
py_pred = X_test @ clr.coef_ + clr.intercept_
py_pred[:5]

array([17.56231959, 18.92593785, 26.14022707, 22.51835039, 22.76141885])

In [8]:
clr.coef_.dtype, clr.intercept_.dtype

(dtype('float64'), dtype('float64'))

## With ONNX

With *ONNX*, we would write this operation as follows... We still need to convert everything into single floats = float32.

In [9]:
%load_ext mlprodict

The mlprodict extension is already loaded. To reload it, use:
  %reload_ext mlprodict


In [10]:
from skl2onnx.algebra.onnx_ops import OnnxMatMul, OnnxAdd
import numpy

onnx_fct = OnnxAdd(OnnxMatMul('X', clr.coef_.astype(numpy.float32)),
                   numpy.array([clr.intercept_]),
                   output_names=['Y'])
onnx_model32 = onnx_fct.to_onnx({'X': X_test.astype(numpy.float32)},
                              dtype=numpy.float32)

# add -l 1 if nothing shows up
%onnxview onnx_model32

The next line uses a python runtime to compute the prediction.

In [11]:
from mlprodict.onnxrt import OnnxInference
oinf = OnnxInference(onnx_model32)
ort_pred = oinf.run({'X': X_test})['Y']
ort_pred[:5]

array([17.56232162, 18.92593974, 26.1402286 , 22.5183519 , 22.76142049])

And here is the same with [onnxruntime](https://github.com/microsoft/onnxruntime)...

In [12]:
oinf = OnnxInference(onnx_model32, runtime="onnxruntime1")
ort_pred = oinf.run({'X': X_test.astype(numpy.float32)})['Y']
ort_pred[:5]

array([17.56232 , 18.925938, 26.140228, 22.51835 , 22.76142 ],
      dtype=float32)

## With double instead of single float

[ONNX](https://onnx.ai/) was originally designed for deep learning which usually uses floats but it does not mean cannot be used. Every number is converted into double floats.

In [13]:
onnx_fct = OnnxAdd(OnnxMatMul('X', clr.coef_.astype(numpy.float64)),
                   numpy.array([clr.intercept_]),
                   output_names=['Y'])
onnx_model64 = onnx_fct.to_onnx({'X': X_test.astype(numpy.float64)},
                              dtype=numpy.float64)

And now the *python* runtime...

In [14]:
oinf = OnnxInference(onnx_model64)
ort_pred = oinf.run({'X': X_test})['Y']
ort_pred[:5]

array([17.56231959, 18.92593785, 26.14022707, 22.51835039, 22.76141885])

And the *onnxruntime* version of it, not fully supportive of double yet...

In [15]:
try:
    oinf = OnnxInference(onnx_model64, runtime="onnxruntime1")
    ort_pred = oinf.run({'X': X_test.astype(numpy.float64)})['Y']
    ort_pred[:5]
except RuntimeError as e:
    print(e)

[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for the node Add:Add(7)


## And now the GaussianProcess

This shows a case

In [16]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct
gau = GaussianProcessRegressor(alpha=10, kernel=DotProduct())
gau.fit(X_train, y_train)

GaussianProcessRegressor(alpha=10, copy_X_train=True,
                         kernel=DotProduct(sigma_0=1), n_restarts_optimizer=0,
                         normalize_y=False, optimizer='fmin_l_bfgs_b',
                         random_state=None)

In [17]:
from skl2onnx import to_onnx
onnxgau32 = to_onnx(gau, X_train.astype(numpy.float32), dtype=numpy.float32)
oinf32 = OnnxInference(onnxgau32, runtime="python")
ort_pred32 = oinf32.run({'X': X_test.astype(numpy.float32)})['GPmean']
numpy.squeeze(ort_pred32)[:25]

array([17.1875 , 20.875  , 25.21875, 22.125  , 23.5625 , 21.78125,
       27.5625 , 35.28125, 26.625  , 16.28125, 29.9375 , 19.0625 ,
       20.125  , 22.875  , 14.4375 , 26.03125, 10.75   , 17.9375 ,
       33.3125 , 13.8125 , 17.75   , 13.21875, 18.     , 32.375  ,
       19.5    ], dtype=float32)

In [18]:
onnxgau64 = to_onnx(gau, X_train.astype(numpy.float64), dtype=numpy.float64)
oinf64 = OnnxInference(onnxgau64, runtime="python")
ort_pred64 = oinf64.run({'X': X_test.astype(numpy.float64)})['GPmean']
numpy.squeeze(ort_pred64)[:25]

array([17.19598599, 20.1674439 , 25.21350425, 21.91513404, 23.18918719,
       21.58785117, 27.5952068 , 35.27743119, 26.57239929, 16.64168282,
       30.38190087, 19.00728743, 20.23123449, 22.54883124, 14.22963182,
       25.65298604, 11.08007669, 18.23620865, 33.34967906, 13.29999159,
       17.57056243, 12.51766272, 17.78160086, 31.98265178, 19.19480978])

The differences between the predictions for single floats and double floats...

In [19]:
numpy.sort(numpy.sort(numpy.squeeze(ort_pred32 - ort_pred64)))[-5:]

array([0.75621721, 0.76835317, 0.76918606, 0.83738951, 0.85145071])

Who's right or wrong... The differences between the predictions with the original model...

In [20]:
pred = gau.predict(X_test.astype(numpy.float64))

In [21]:
numpy.sort(numpy.sort(numpy.squeeze(ort_pred32 - pred)))[-5:]

array([0.75621721, 0.76835317, 0.76918606, 0.83738951, 0.85145071])

In [22]:
numpy.sort(numpy.sort(numpy.squeeze(ort_pred64 - pred)))[-5:]

array([0., 0., 0., 0., 0.])

Double predictions clearly wins.

In [23]:
# add -l 1 if nothing shows up
%onnxview onnxgau64

## Saves...

Let's keep track of it.

In [24]:
with open("gpr_dot_product_boston_32.onnx", "wb") as f:
    f.write(onnxgau32.SerializePartialToString())
from IPython.display import FileLink
FileLink('gpr_dot_product_boston_32.onnx')

C:\xavierdupre\__home_\GitHub\mlprodict\_doc\notebooks\gpr_dot_product_boston_32.onnx

In [25]:
with open("gpr_dot_product_boston_64.onnx", "wb") as f:
    f.write(onnxgau64.SerializePartialToString())
FileLink('gpr_dot_product_boston_64.onnx')

C:\xavierdupre\__home_\GitHub\mlprodict\_doc\notebooks\gpr_dot_product_boston_64.onnx